# 02 Exploratory Data Analysis (EDA)

In [2]:
import pandas as pd

In [3]:
# Show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Optional: widen the column width and overall display width
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 0)

In [4]:
# Plot Energy Use over Time
# (Convert Time from Unix to Datetime before)

In [5]:
# Plot: Energy Use vs Features (Check Dependency, likely Temperature) - Scatter

In [6]:
# Plot: Energy Use vs Weather Conditions - Bars or Box

In [7]:
# Plot: Distribution Cloud Cover KDE

In [8]:
# Plot: Energy Consumption over Time (Multiple Lines for each feature)

In [9]:
# Plot: Weather feature Distribution over Time

In [10]:
# Proportiion of used applications